# Рекуррентные нейронные сети

В предыдущем модуле мы рассмотрели семантическое представление слов, но при этом мы использовали агрегированное значение слов в предложении и не учитывали **порядок слов** - поскольку операция агрегации, следующая за эмбеддингом, удаляет эту информацию из исходного текста. Модели, не учитывающие порядок слов, не могут решать более сложные или задачи, такие как генерация текста или ответы на вопросы.

Чтобы получить значение текстовой последовательности с учётом порядка слов, мы будем использовать архитектуру нейронной сети, называемую **рекуррентная нейронная сеть**, или RNN. При использовании RNN мы пропускаем наше предложение через сеть по одному токену за раз, и каждый раз сеть вычисляет некоторое **состояние**, которое мы затем снова передаем в сеть со следующим токеном.

![Image showing an example recurrent neural network generation.](images/rnn.png)

Рассмотрим задачу классификации с помощью RNN. Пусть $X_0,\dots,X_n$ - входная последовательность токенов. RNN создает последовательность блоков нейронной сети и обучает эту последовательность от начала до конца с помощью обратного распространения. Каждый сетевой блок принимает пару $(X_i,S_i)$ в качестве входных данных и в результате выдает $S_{i+1}$. Конечное состояние $S_n$ или выходной $Y_n$ поступают в линейный классификатор для получения результата. Все сетевые блоки имеют одинаковые веса и обучаются от начала до конца с использованием одного прохода обратного распространения.

> На рисунке выше показана рекуррентная нейронная сеть в развернутом виде (слева) и в более компактном рекуррентном представлении (справа). Важно понимать, что все ячейки RNN имеют одинаковые **разделяемые веса**.

Поскольку векторы состояний $S_0,\dots,S_n$ передаются через сеть, RNN может обнаруживать зависимости между словами, т.е. определённые паттерны. Например, когда слово *not* появляется где-то в последовательности, оно может запомнить это в векторе состояния.

Внутри каждая ячейка RNN содержит две матрицы весов: $W_H$ и $W_I$, а также вектор сдвига $b$. На каждом шаге RNN с входом $X_i$ и входном состоянии $S_i$, выходное состояние вычисляется как $S_{i+1} = f(W_H\times S_i + W_I\times X_i+b)$, где $f$ — функция активации (часто $\tanh$).

> Для таких задач, как генерация текста или машинный перевод, мы также хотим получить некоторое выходное значение на каждом шаге RNN. В этом случае также существует другая матрица $W_o$, и выходные данные вычисляются как $Y_i=f(W_o\times S_i+b_o)$.

Давайте посмотрим, как рекуррентные нейронные сети могут использоваться в задаче классификации датасета новостей.

In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

При обучении больших моделей становится проблемой выделение памяти, в особенности памяти GPU, поскольку её размер обычно ограничен. Нам, возможно, придется экспериментировать с различными размерами минибатчей, чтобы данные помещались в память нашего графического процессора.

> **Замечание**: Некоторые версии драйверов NVidia могут неправильно освобождать память после обучения модели. Мы в одном ноутбуке проводим несколько экспериментов, поэтому в некоторых случаях при возможно исчерпание памяти, особенно если вы делаете параллельно какие-то дополнительные эксперименты. Если такое случится - перезапустите Jupyter Kernel.

In [2]:
batch_size = 16
embed_size = 64

## Простой рекуррентный классификатор

Для классической RNN сети каждый рекуррентнай блок представляет собой простую полносвязную сеть, которая принимает входной вектор и вектор состояния и создает новый вектор состояния. В Keras такой блок называется `SimpleRNN`.

Мы можем подавать на вход RNN токены в one hot encoding, это не очень хорошая идея из-за их высокой размерности. Поэтому мы будем использовать слой Embedding для снижения размерности векторов слов, за которым следует слой RNN и, наконец, классификатор `Dense`.

> **Примечание**: В тех случаях, когда размерность не так высока, например, при использовании токенизации на уровне символов, может иметь смысл передавать one-hot-encoded токены непосредственно в ячейку RNN.

In [3]:
vocab_size = 20000

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=vocab_size,
    input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 64)          1280000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 16)                1296      
                                                                 
 dense (Dense)               (None, 4)                 68        
                                                                 
Total params: 1,281,364
Trainable params: 1,281,364
Non-trainable params: 0
_________________________________________________________________


> **Примечание:** Мы используем необученный embedding-слой, но для достижения лучших результатов мы можем использовать предварительно обученные с помощью Word2Vec вектора.

Давайте обучим наш RNN. В целом рекуррентные сети довольно трудно обучать, потому что после развёртывания всех блоков RNN по длине последовательности у нас может получиться достаточно длинная цепочка для обратного распространения. Рекомендуется выбрать меньшую скорость обучения и обучить сеть на большем наборе данных для получения хороших результатов. Это может занять довольно много времени, поэтому использование графического процессора предпочтительнее.

Чтобы ускорить процесс, в этом примере мы будем обучать модель RNN только на заголовках новостей, опуская полный текст новости. Вы можете попробовать обучение с описанием и посмотреть, как поменяется точность.

In [4]:
@tf.function
def extract_title(x):
    return x['title']

@tf.function
def tupelize_title(x):
    return (extract_title(x),x['label'])

print('Training vectorizer')
vectorizer.adapt(ds_train.take(2000).map(extract_title))

Training vectorizer


2022-12-03 20:27:19.114197: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-03 20:27:19.776663: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [5]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize_title).batch(batch_size),validation_data=ds_test.map(tupelize_title).batch(batch_size))

7500/7500 [==============================] - 48s 6ms/step - loss: 0.6692 - acc: 0.7542 - val_loss: 0.5853 - val_acc: 0.7888


> **Обратите внимание**, что точность, скорее всего, будет ниже, чем в примерах ранее, поскольку мы тренируемся только на  заголовках новостей.

## Последовательности переменной длины и RNN 

Мы говорили, что слой `TextVectorization` будет автоматически дополнять последовательности переменной длины в минибатче нулевыми токенами. Оказывается, эти токены также принимают участие в обучении, и они могут усложнить сходимость модели. Это особенно верно в случае с RNN, поскольку таким токенам соответствуют развёрнутые рекуррентные ячейки.

Есть несколько подходов, которые можно использовать для уменьшения проблем с паддингом. Один из них, который мы уже упоминали - это сортировка данных по длине последовательности и группировка всех последовательностей по размеру. Это можно сделать с помощью функции `tf.data.experimental.bucket_by_sequence_length` (см. [документация](https://www.tensorflow.org/api_docs/python/tf/data/experimental/bucket_by_sequence_length)). 

Другой подход заключается в использовании **маскирования** (*masking*). В Keras некоторые слои поддерживают дополнительный вход, который показывает, какие токены следует учитывать при обучении. Чтобы включить маскирование в нашу модель, мы можем либо добавить отдельный слой `Masking` ([документация](https://keras.io/api/layers/core_layers/masking/)), либо указать параметр `mask_zero=True` при описании слоя `Embedding`.

> **Примечание**: Обучение такой сети может занять существенное время, поэтому мы тренируем одну эпоху. Не стесняйтесь прервать обучение в любое время, если у вас закончится терпение. Что вы также можете сделать, так это ограничить объем данных, используемых для обучения, добавив `.take(...)` для ограничения используемого количества записей в `ds_train` и `ds_test`.

In [6]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size,embed_size,mask_zero=True),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

7500/7500 [==============================] - 87s 11ms/step - loss: 0.5030 - acc: 0.8300 - val_loss: 0.4155 - val_acc: 0.8666


Теперь, когда мы используем маскирование, мы можем обучить модель нам всем наборе данных - на заголовках и самих новостях.

> **Примечание**: Вы заметили, что мы используем векторизатор, обученный на заголовках новостей, а не на всем теле статьи? Потенциально это может привести к тому, что некоторые токены будут проигнорированы, поэтому лучше переобучить векторизатор. Тем не менее, это может иметь только очень небольшой эффект, поэтому мы будем придерживаться предыдущего предварительно обученного векторизатора для простоты.

## LSTM: Долговременная кратковременная память

Одной из основных проблем RNN являются **исчезающие градиенты**. RNN-цепочки могут быть довольно длинными, и при распространении градиентов обратно от последнего уровня сети к первому во время обратного распространения могут возникать сложности. Из-за этого классическая RNN-сеть не может выучить взаимосвязи между удаленными друг от друга токенами. Одним из способов избежать этой проблемы является введение **явного управления состоянием** с помощью **вентилей** (*gates*). Двумя наиболее распространенными архитектурами, которые вводят вентили, являются **долговременная кратковременная память** (LSTM) и **вентильный релейный блок** (GRU). Мы здесь рассмотрим LSTM.

![Изображение, показывающее пример ячейки долговременной памяти](images/long-short-term-memory-cell.svg)

Сеть LSTM организована аналогично RNN, но есть два состояния, которые передаются от слоя к слою: фактическое состояние $c$ и скрытый вектор $h$. На каждом блоке скрытый вектор $h_{t-1}$ объединяется с входным $x_t$, и вместе они управляют тем, что происходит с состоянием $c_t$ и выходом $h_{t}$ с помощью **вентилей**. Каждый вентиль представляет собой нейросеть с сигмовидной активацией (выход в диапазоне $[0,1]$), которую можно рассматривать как побитовую маску при умножении на вектор состояния. LSTM имеют следующие вентили (слева направо на рисунке выше):

* вентиль **забывания** - определяет, какие компоненты вектора $c_{t-1}$ нам нужно забыть, а какие - оставить 
* **входной** вентиль - определяет, какую информацию из входного вектора и предыдущего скрытого вектора состояния $h_{t-1}$ необходимо включить в новый вектор состояния $c_t$
* **выходной** вентиль, который принимает новый вектор состояния и решает, какие из его компонентов будут использоваться для получения нового скрытого вектора $h_t$.

Компоненты состояния $c$ можно представлять как флаги, которые можно включать и выключать. Например, когда мы сталкиваемся с именем *Алиса* в последовательности, мы догадываемся, что оно относится к женщине, и поднимаем флаг в состоянии, которое говорит, что у нас есть женское существительное в предложении. Когда мы в дальнейшем столкнемся со словами *и Том*, мы поднимем флаг, который говорит, что у нас есть существительное множественного числа. Таким образом, манипулируя состоянием, мы можем отслеживать грамматические свойства предложения.

> **Примечание**: Вот отличные статьи для понимания внутреннего устройства LSTM: [на русском](https://sysblok.ru/knowhow/mama-myla-lstm-kak-ustroeny-rekurrentnye-nejroseti-s-dolgoj-kratkosrochnoj-pamjatju/) и [на английском](https://colah.github.io/posts/2015-08-Understanding-LSTM/).

Хотя внутренняя структура ячейки LSTM может показаться сложной, Keras скрывает эту реализацию внутри слоя `LSTM`, поэтому единственное, что нам нужно сделать в приведенном выше примере, это заменить название слоя:

In [7]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.LSTM(8),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(8),validation_data=ds_test.map(tupelize).batch(8))

15000/15000 [==============================] - 176s 12ms/step - loss: 0.5957 - acc: 0.7602 - val_loss: 0.3923 - val_acc: 0.8805


> **Обратите внимание**, что обучение LSTM также довольно медленное, и вам может показаться, что в начале тренировки не сильно повысится точность. Возможно, вам придется продолжать обучение в течение некоторого времени, чтобы достичь хорошей точности, и увеличить количество эпох.

## Двунаправленные и многослойные RNN

Мы рассмотрели рекуррентные сети, которые идут по последовательности от начала до конца, слева направо. Это кажется естественным, поскольку следует тому же направлению, в котором мы читаем или слушаем речь. Однако для сценариев, требующих произвольного доступа к входной последовательности, имеет смысл строить рекуррентные сети, работающие в обоих направлениях. RNN, которые позволяют выполнять вычисления в обоих направлениях, называются **двунаправленными**, и их можно создать, обернув рекуррентный слой специальным «двунаправленным» слоем.

> **Примечание**: Обёртка `Bidirectional` делает две копии вложенного в него слоя, и устанавливает свойство `go_backwards` одной из этих копий в `True`, заставляя его идти в противоположном направлении по последовательности.

Рекуррентные сети, однонаправленные или двунаправленные, захватывают шаблоны в последовательности и сохраняют их в векторах состояний или возвращают в качестве выходных данных. Как и в случае со сверточными сетями, мы можем построить еще один рекуррентный слой, следующий за первым, чтобы распознавать шаблоны более высокого уровня, построенные из шаблонов более низкого уровня, извлеченных первым слоем. Это приводит нас к понятию **многослойной RNN**, которая состоит из двух или более рекуррентных слоёв, где выход предыдущего слоя передается следующему уровню в качестве входа.

![Image showing a Multilayer long-short-term-memory- RNN](images/multi-layer-lstm.jpg)

Keras делает построение таких сетей легкой задачей, потому что нам просто нужно добавить нужное количество повторяющихся слоев в модель. Для всех слоев, кроме последнего, нам нужно указать параметр `return_sequences=True`, потому что нам нужно, чтобы слой возвращал все промежуточные состояния, а не только конечное состояние рекуррентного вычисления.

Давайте построим двухслойный двунаправленный LSTM для нашей задачи классификации.

> **Примечание**: обучение такой сети занимает довольно много времени, но в результате мы получим самую высокую точность, которую мы видели до сих пор. Так что, возможно, стоит подождать и увидеть результат.

In [8]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, 128, mask_zero=True),
    keras.layers.Bidirectional(keras.layers.LSTM(64,return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),    
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

7500/7500 [==============================] - 651s 85ms/step - loss: 0.3519 - acc: 0.8790 - val_loss: 0.3115 - val_acc: 0.8964


## RNN для других задач

До сих пор мы фокусировались на использовании RNN для классификации последовательностей текста. Но они могут справиться со многими другими задачами, такими как генерация текста и машинный перевод — мы рассмотрим эти задачи в следующем блоке.